In [2]:
import pandas as pd
import numpy as np
from gamspy import Container, Set, Parameter, Variable, Equation, Model, Sum, Sense

In [3]:
def binomial (n, p, size):
    """
    Generate random numbers following a binomial distribution.

    Parameters:
    - n:    Number of trials (number of times an event is repeated).
    - p:    Probability of success for each trial.
    - size: Number of times to repeat the binomial experiment.

    Returns:
    - An array of random numbers representing the number of successes in each experiment.
    """
    return np.random.binomial(n, p, size)

def randint (low, high, size):
    """
    Generate random integers from a discrete uniform distribution.

    Parameters:
    - low:  The minimum (inclusive) value of the range.
    - high: The maximum (exclusive) value of the range.
    - size: The number of random integers to generate.

    Returns:
    - An array of random integers within the specified range.
    """
    return np.random.randint (low, high, size)

# QUESTION 1
There are n products to be produced $\newline$
There are m suppliers which provide the parts to produce the products $\newline$
A product i needs $a_{ij}$ parts ($a_{ij >= 0}$, $i = 1\rightarrow n$, $j=1\rightarrow m $) $\newline$
The preorder cost for part $j$ is $b_j$ $\newline$ 
The demand for the products is random vector $D = (D_1,..., D_n)$ $\newline$
Cost additionaly $l_i$ to satisfy a unit of demand for product $i$, and unit selling price of that unit is $q_i$ $\newline$
The number of parts ordered is $x_j$, the number of units produced is $z_i$, the numbers of remaining parts unused is $y_j$


### Second stage
The second stage is mentioned first to emphasize the importance of understanding the uncertainty in case of solving a stochastic problem. This approach aligns with the principles of stochastic programming, where the second-stage decisions are made after observing the realization of random data and are contingent on the first-stage decisions. $\newline$
In this stage, we want to minimize the cost of production. Specifically, we need to find the optimal cost of filling an additional portion of demand while reselling the salvage parts to minimize the loss of not using the parts. $\newline$
$$ 
min_{x,y} \text { } Z=\sum^n_{i=1} (l_i-q_i)z_i - \sum^m_{j=1}s_jy_j \newline \text {subject to} \newline
y_j=x_j- \sum_{i=1}^n a_{ij}z_i, j=1,...,m \newline
0 \leq z_i \leq d_i, i=1,...,n \newline
y_j \geq 0, j=1,...,m
$$

### First stage
In this stage, $x$ needs to be decided before a realization of the demand $D$, which makes it a $\textit {here and now}$ decisions $\newline$
Moreover, the expected value of $Z$ should also be taken into account. To be specific, that is the optimal value $Q(x)$ of the second stage ($Q(x) = E[Z(z,y)]$) $\newline$
Therefore, we yield this problem below: 
$$min \text { } g(x,y,z) = b^Tx + Q(x) = b^Tx + E[Z(z,y)]$$

### Big problem
We need to solve this
$$
min \text { } b^Tx + \sum^S_{i=1} p_s[(l_i-q_i)^Tz_i - s^T_iy_i] \newline
\text {subject to} \newline
y \geq 0 \newline
0 \leq z \leq d \newline
x \geq 0 \newline
$$

In [38]:
n   = 8           # number of products
m   = 5           # number of parts to be ordered
S   = 2           # number of scenarios
p_s = [0.5, 0.5]  # probability of each scenario / density

# D = np.array([binomial (10, 0.5, n),
#               binomial (10, 0.5, n)]) # demand for each product in each scenario matrix [n x S]

b = randint(5, 10, m) # preorder cost for each part

A = []
for i in range(n):
    A.append(np.array(randint(1,10,m)))

A = np.array(A) # bill of materials matrix [m x n]
# A = np.transpose(A)

s = randint (1, 10, m)       # salvage values
l = randint (100, 200, n)    # additional costs
q = randint (1000, 1400, n)  # unit selling prices

c = l - q #cost coefficients

D

array([[3, 5, 0, 6, 4, 4, 7, 6],
       [6, 6, 7, 5, 5, 6, 5, 5]])

In [13]:
print('Preorder cost: ', b)
print('Material usage:\n', A)
print('Salvage value: ', s)
print('Additional producing cost: ', l)
print('Selling prices: ', q)
print('Cost coefficient:', c)
# print('Demand:\n', D)

Preorder cost:  [5 7 5 7 8]
Material usage:
 [[5 2 4 8 7]
 [5 1 3 4 7]
 [2 7 7 7 8]
 [5 8 1 6 9]
 [1 1 9 8 3]
 [2 1 5 8 8]
 [1 5 2 9 5]
 [6 8 4 5 6]]
Salvage value:  [8 2 1 3 4]
Additional producing cost:  [198 147 189 185 110 174 173 131]
Selling prices:  [1285 1041 1355 1343 1216 1047 1354 1067]
Cost coefficient: [-1087  -894 -1166 -1158 -1106  -873 -1181  -936]
Demand:
 [[5 5 4 7 6 6 7 8]
 [8 5 6 5 1 6 3 5]]


In [294]:
# df_parts_usage = []
# for i in range(m):
#     df_parts_usage.append([PARTS_NAME[i]])
#     for j in range(n):
#         df_parts_usage[i].append(A[i][j])

# df_parts_usage = pd.DataFrame(
#     data = df_parts_usage,
#     columns=[''] + PRODUCTS_NAME
# ).set_index('')

# df_parts_usage

In [295]:
m = Container()


In [296]:
i = Set(
    container=m,
    name='products',
    description='products of the firm',
    records=PRODUCTS_NAME
)
i.records

,uni,element_text
0,product 1,
1,product 2,
2,product 3,
3,product 4,
4,product 5,
5,product 6,
6,product 7,
7,product 8,


In [297]:
j = Set(
    container=m,
    name='parts',
    description='Parts to build the products',
    records=PARTS_NAME
)
j.records

,uni,element_text
0,part 1,
1,part 2,
2,part 3,
3,part 4,
4,part 5,


In [298]:
b = Parameter(
    container=m,
    name='Preorder_price',
    domain=j,
    description='Price to preorder a part',
    records=B
)
b.records

,parts,value
0,part 1,9.0
1,part 2,6.0
2,part 3,9.0
3,part 4,7.0
4,part 5,6.0


In [299]:
a = Parameter (
    container=m,
    name='parts_usages',
    domain=[i, j],
    description='Numbers of parts needed for n products',
    records= A
)
a.records

,products,parts,value
0,product 1,part 1,2.0
1,product 1,part 2,2.0
2,product 1,part 3,6.0
3,product 1,part 4,9.0
4,product 1,part 5,7.0
5,product 2,part 1,4.0
6,product 2,part 2,2.0
7,product 2,part 3,9.0
8,product 2,part 4,8.0
9,product 2,part 5,1.0


In [300]:
sc = Set(
    container=m,
    name='Scenario',
    description="Two scenario of the demand",
    records = ['Scenario 1', 'Scenario 2']
)

d = Parameter(
    container=m,
    name='Demand',
    domain=[sc,i],
    description="Demand for products",
    records = D
)

d.records

,Scenario,products,value
0,Scenario 1,product 1,3.0
1,Scenario 1,product 2,5.0
2,Scenario 1,product 3,4.0
3,Scenario 1,product 4,6.0
4,Scenario 1,product 5,4.0
5,Scenario 1,product 6,7.0
6,Scenario 1,product 7,7.0
7,Scenario 1,product 8,5.0
8,Scenario 2,product 1,4.0
9,Scenario 2,product 2,4.0


In [301]:
s = Parameter(
    container=m,
    name='Salvage_price',
    domain=j,
    records=S
)
s.records

,parts,value
0,part 1,6.0
1,part 2,2.0
2,part 3,4.0
3,part 4,6.0
4,part 5,3.0


In [302]:
l = Parameter(
    container=m,
    name='Addition_producing_cost',
    domain=i,
    records = L
)
l.records

,products,value
0,product 1,132.0
1,product 2,180.0
2,product 3,138.0
3,product 4,114.0
4,product 5,132.0
5,product 6,132.0
6,product 7,192.0
7,product 8,130.0


In [303]:
q = Parameter(
    container=m,
    name='Selling_price',
    domain=i,
    records=Q
)
q.records

,products,value
0,product 1,1373.0
1,product 2,1377.0
2,product 3,1326.0
3,product 4,1290.0
4,product 5,1194.0
5,product 6,1353.0
6,product 7,1022.0
7,product 8,1279.0


In [304]:
x = Variable(
    container=m,
    name='x',
    domain=i,
    description='Number of preordered parts'
)

z = Variable(
    container=m,
    name='z',
    domain=i,
    description='Number of produced products'
)

y = Variable(
    container=m,
    name='y',
    domain=j,
    description='Number of unused parts'
)

In [305]:
con_1_1 = Equation(
    container=m,
    name='Constraint_1_1',
    domain=i,
    definition=x>=0
) 

con_1_2 = Equation(
    container=m,
    name='Constraint_1_2',
    domain=j,
    definition=y>=0
) 

con_1_3 = Equation(
    container=m,
    name='Constraint_1_3',
    domain=i,
    definition=z>=0
) 

con_1_4 = Equation(
    container=m,
    name='Constraint_1_4',
    domain=i
) 
con_1_4[i] = x - Sum(j, a[i,j]*z[i])

con_1_5 = Equation(
    container=m,
    name='Constraint_1_5',
    domain=i,
    definition= z <= d
) 

$$
min \text { } b^Tx + \sum^S_{i=1} p_s[(l_i-q_i)^Tz_i - s^T_iy_i]$$

In [306]:
obj = b*x + Sum(sc, 0.5*(l[i]-q[i]*q[i]-s[i]*y[i]))

In [307]:
production = Model(
    container=m,
    name='production',
    equations=m.getEquations(),
    problem='LP',
    sense=Sense.MIN,
    objective=obj
)

In [308]:
import sys
production.solve(output=sys.stdout)

--- Job _job_756a863f-0277-41ac-965b-cd52ef0d4a53.gms Start 12/03/23 13:40:15 45.4.0 19dc3313 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\DUC_AN\.conda\envs\gamspy\Lib\site-packages\gamspy_base\gmsprmNT.txt
--- GAMS Parameters defined
    LP CPLEX
    MIP CPLEX
    RMIP CPLEX
    NLP CONOPT
    MCP PATH
    MPEC NLPEC
    RMPEC CONVERT
    CNS CONOPT
    DNLP CONOPT
    RMINLP CONOPT
    MINLP SBB
    QCP CONOPT
    MIQCP SBB
    RMIQCP CONOPT
    EMP CONVERT
    Input C:\Users\DUC_AN\AppData\Local\Temp\tmpmdbtkhq3\_job_756a863f-0277-41ac-965b-cd52ef0d4a53.gms
    Output C:\Users\DUC_AN\AppData\Local\Temp\tmpmdbtkhq3\_job_756a863f-0277-41ac-965b-cd52ef0d4a53.lst
    Save C:\Users\DUC_AN\AppData\Local\Temp\tmpmdbtkhq3\_save_c1dbcfc7-24b8-44d5-af47-fc611ebe2c46.g00
    ScrDir C:\Users\DUC_AN\AppData\Local\Temp\tmpmdbtkhq3\225a\
    SysDir C:\Users\DUC_AN\.conda\envs\gamspy\Lib\site-packages\gamspy_base\
    CurDir C:\Users\DUC_AN\AppData\Local\Temp\tmpmdbtkhq3\
    LogOption 

GamsExceptionExecution: 

================================================================================
Error Summary
================================================================================
  30  Constraint_1_1 .. x =g= 0;
****                  $148  $148
**** 148  Dimension different - The symbol is referenced with more/less
****         indices as declared

GAMS return code not 0 (2), check C:\Users\DUC_AN\AppData\Local\Temp\tmpmdbtkhq3\_job_756a863f-0277-41ac-965b-cd52ef0d4a53.lst for more details